# Schema Understanding and Table Joins

This notebook focuses on constructing an analytics-ready dataset by joining
fact and dimension tables from the data warehouse.

The dataset follows a **star schema**, where:
- Fact tables store measurable business events
- Dimension tables provide descriptive attributes

LEFT JOINs are used to preserve all fact records and avoid data loss.

In [1]:
# Standard library for file system operations
import os

# Pandas for data loading and table joins
import pandas as pd

### Defining a Robust Data Loading Function

The function `load_csv()` is defined to facilitate reproducible and flexible data ingestion. It accepts the following parameters:

- `DATA_DIR`: The directory path containing the data warehouse files.
- `filename`: The specific CSV file to be loaded.
- `encoding`: The character encoding to be used (default: `latin1`).



In [2]:
def load_csv(DATA_DIR, filename, encoding="latin1"):
    return pd.read_csv(os.path.join(DATA_DIR, filename), encoding=encoding)

In [3]:
# Try both possible data directory paths
possible_dirs = ["../data/raw", "data/raw"]
DATA_DIR = None

# Find the first existing data directory
for d in possible_dirs:
    # Check if the directory exists
    if os.path.isdir(d):
        DATA_DIR = d
        break

# Raise an error if no data directory is found
if DATA_DIR is None:
    raise FileNotFoundError("Could not find the data/raw directory. Checked: {}".format(possible_dirs))

# Confirm the resolved data directory
DATA_DIR

'../data/raw'

### Loading tables

Loading tables using the load_csv function created earlier.

In [4]:
# Load the main fact table explicitly
fact_sales = load_csv(DATA_DIR, "FactInternetSales.csv", "utf-8-sig")

# Load key dimension tables
dim_product = load_csv(DATA_DIR, "DimProduct.csv")
dim_customer = load_csv(DATA_DIR, "DimCustomer.csv")
dim_date = load_csv(DATA_DIR, "DimDate.csv")

# Display shapes to understand join sizes
fact_sales.shape, dim_product.shape, dim_customer.shape, dim_date.shape

((60398, 26), (606, 27), (18484, 29), (2191, 20))

The fact table is significantly larger than the dimension tables, which is consistent with best practices in dimensional data warehouse design.

##### Why specify encoding when loading CSVs?

Some CSV files in this dataset contain special or non-UTF-8 characters (such as accented letters or symbols) that cause errors when using the default encoding (UTF-8) in pandas. To avoid `UnicodeDecodeError`, we explicitly set `encoding="latin1"` when loading these files. This encoding is more permissive and can handle a wider range of characters found in international datasets.

In the dataset overview notebook, we only listed file names and did not load any CSV content, so encoding was not needed there. If you encounter similar decoding errors when loading files in other notebooks, you should also specify the encoding parameter.

### Normalize column names

In [5]:
# Inspect column names in fact and product dimension tables
print("FactInternetSales columns:")
print(fact_sales.columns.tolist())

print("\nDimProduct columns:")
print(dim_product.columns.tolist())


FactInternetSales columns:
['ProductKey', 'OrderDateKey', 'DueDateKey', 'ShipDateKey', 'CustomerKey', 'PromotionKey', 'CurrencyKey', 'SalesTerritoryKey', 'SalesOrderNumber', 'SalesOrderLineNumber', 'RevisionNumber', 'OrderQuantity', 'UnitPrice', 'ExtendedAmount', 'UnitPriceDiscountPct', 'DiscountAmount', 'ProductStandardCost', 'TotalProductCost', 'SalesAmount', 'TaxAmt', 'Freight', 'CarrierTrackingNumber', 'CustomerPONumber', 'OrderDate', 'DueDate', 'ShipDate']

DimProduct columns:
['ProductKey', 'ProductAlternateKey', 'ProductSubcategoryKey', 'WeightUnitMeasureCode', 'SizeUnitMeasureCode', 'EnglishProductName', 'SpanishProductName', 'FrenchProductName', 'StandardCost', 'FinishedGoodsFlag', 'Color', 'SafetyStockLevel', 'ReorderPoint', 'ListPrice', 'Size', 'SizeRange', 'Weight', 'DaysToManufacture', 'ProductLine', 'DealerPrice', 'Class', 'Style', 'ModelName', 'EnglishDescription', 'StartDate', 'EndDate', 'Status']


By printing the column names we notice an issue.

The issue is that the first column in FactInternetSales is named 'ï»¿ProductKey' instead of 'ProductKey'. This happens because the CSV file likely contains a Byte Order Mark (BOM) at the start.

To solve the issue, when reading the CSV, use the encoding `utf-8-sig` for `FactInternetSales.csv`.

No need to use `encoding="utf-8-sig"` for the other tables unless you see the same BOM issue. If the other tables load correctly and their column names look normal, we can keep using encoding="latin1" or the default.



In [6]:
# Standardize column names by stripping whitespace
fact_sales.columns = fact_sales.columns.str.strip()
dim_product.columns = dim_product.columns.str.strip()
dim_customer.columns = dim_customer.columns.str.strip()
dim_date.columns = dim_date.columns.str.strip()

## Joining FactInternetSales with Dimension Tables

The fact table is incrementally joined with relevant dimension tables
to enrich transactional records with descriptive attributes.

LEFT JOINs ensure that all sales transactions are retained even if
corresponding dimension records are missing.


In [7]:
# Join FactInternetSales with DimProduct to add product attributes
fact_with_product = fact_sales.merge(
    dim_product,
    on="ProductKey",
    how="left"
)

# Validate join result
fact_with_product.shape

(60398, 52)

### Inspect new columns added by DimProduct

In [8]:
# Show columns coming from DimProduct
product_columns = [col for col in fact_with_product.columns if col not in fact_sales.columns]
product_columns


['ProductAlternateKey',
 'ProductSubcategoryKey',
 'WeightUnitMeasureCode',
 'SizeUnitMeasureCode',
 'EnglishProductName',
 'SpanishProductName',
 'FrenchProductName',
 'StandardCost',
 'FinishedGoodsFlag',
 'Color',
 'SafetyStockLevel',
 'ReorderPoint',
 'ListPrice',
 'Size',
 'SizeRange',
 'Weight',
 'DaysToManufacture',
 'ProductLine',
 'DealerPrice',
 'Class',
 'Style',
 'ModelName',
 'EnglishDescription',
 'StartDate',
 'EndDate',
 'Status']

In [9]:
# Preview selected business-relevant columns
fact_with_product[[
    "SalesOrderNumber",
    "ProductKey",
    "EnglishProductName",
    "Color",
    "SalesAmount",
    "OrderQuantity"
]].head()


,SalesOrderNumber,ProductKey,EnglishProductName,Color,SalesAmount,OrderQuantity
0,SO43697,310,"Road-150 Red, 62",Red,3578.2700,1
1,SO43698,346,"Mountain-100 Silver, 44",Silver,3399.9900,1
2,SO43699,346,"Mountain-100 Silver, 44",Silver,3399.9900,1
3,SO43700,336,"Road-650 Black, 62",Black,699.0982,1
4,SO43701,346,"Mountain-100 Silver, 44",Silver,3399.9900,1


In [10]:
dim_customer.shape

(18484, 29)

In [11]:
# Join with customer dimension
fact_with_product_customer = fact_with_product.merge(
    dim_customer,
    on="CustomerKey",
    how="left"
)

# Validate join
assert fact_with_product_customer.shape[0] == fact_with_product.shape[0], \
    "Row count mismatch after joining DimCustomer"

fact_with_product_customer.shape


(60398, 80)

## Resulting Analytics Dataset

The final joined dataset represents a denormalized, analytics-ready table.
This dataset will be used in subsequent notebooks for:

- Exploratory Data Analysis (EDA)
- Feature engineering
- Machine learning model development
